# Data quality and consistency checks 

## Contents

## 1. First checks - Including Sweetviz Data Profile!
## 2. Data Prep:
### 2.1. Data quality checks
### 2.2 Consistency checks
## 3. Outlier check
## 4. Duplicates check
## 5. Timeliness
## 6. Data Profile
## 7. Further inconsistency check:
### Check unique values in string variables 

In [7]:
# Importing libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import re
import pydantic
import sweetviz as sv

In [2]:
# Importing new data set
path = r'/Users/sophie/Desktop/CareerFoundry /10 2023 missing migrants'
path

'/Users/sophie/Desktop/CareerFoundry /10 2023 missing migrants'

## 1. First checks

In [5]:
mm_ds_original = pd.read_excel(os.path.join(path, 'Data', 'original data', 'from Website', 'Missing_Migrants_Global_Figures_allData_1.xlsx'))
pd.options.display.max_columns = None
mm_ds_original.head()

,Main ID,Incident ID,Incident Type,Region of Incident,Website Date,Incident year,Reported Month,Number of Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Region of Origin,Cause of Death,Country of Origin,Migration route,Location of death,Information Source,Coordinates,UNSD Geographical Grouping,Article title,Source Quality,URL
0,2014.MMP00001,2014.MMP00001,Incident,North America,2014-01-06,2014,January,1.0,NaN,1,NaN,NaN,1.0,NaN,Central America,Mixed or unknown,Guatemala,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.650259, -110.366453",Northern America,NaN,5.0,http://humaneborders.info/
1,2014.MMP00002,2014.MMP00002,Incident,North America,2014-01-12,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.59713, -111.73756",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,NaN
2,2014.MMP00003,2014.MMP00003,Incident,North America,2014-01-14,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.94026, -113.01125",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,NaN
3,2014.MMP00004,2014.MMP00004,Incident,North America,2014-01-16,2014,January,1.0,NaN,1,NaN,NaN,1.0,NaN,Central America,Violence,Mexico,US-Mexico border crossing,"near Douglas, Arizona, USA","Ministry of Foreign Affairs Mexico, Pima Count...","31.506777, -109.315632",Northern America,Mexico condemns the use of force by a border p...,5.0,http://bit.ly/1qfIw00
4,2014.MMP00005,2014.MMP00005,Incident,Europe,2014-01-16,2014,January,1.0,0.0,1,2.0,NaN,1.0,NaN,Northern Africa,Harsh environmental conditions / lack of adequ...,Sudan,NaN,Border between Russia and Estonia,EUBusiness (Agence France-Presse),"59.1551, 28",Northern Europe,Sudanese migrant freezes to death in illegal E...,1.0,http://bit.ly/1rTFTjR


In [6]:
mm_ds_original.tail()

,Main ID,Incident ID,Incident Type,Region of Incident,Website Date,Incident year,Reported Month,Number of Dead,Minimum Estimated Number of Missing,Total Number of Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Region of Origin,Cause of Death,Country of Origin,Migration route,Location of death,Information Source,Coordinates,UNSD Geographical Grouping,Article title,Source Quality,URL
13488,2023.MMP1046,2023.MMP1046,Incident,North America,2023-10-11,2023,October,1.0,NaN,1,2.0,1.0,NaN,NaN,Western Africa,Sickness / lack of access to adequate healthcare,Guinea,US-Mexico border crossing,Open air detention site near San Ysidro border...,"NBCS San Diego, San Diego Union Tribune","32.54156785, -117.061009",Northern America,CBP confirms death of migrant camping between...,3.0,https://web.archive.org/web/*/https://www.nbcs...
13489,2023.MMP1047,2023.MMP1047,Incident,Europe,2023-10-13,2023,October,7.0,NaN,7,16.0,NaN,NaN,1.0,Western Asia,Vehicle accident / death linked to hazardous t...,Mixed,NaN,A94 motorway exit between Ampfing and Waldkrai...,"The Telegraph, The Street Journal","48.24627651, 12.410189",Western Europe,Seven dead as minibus believed to be smugglin...,3.0,https://web.archive.org/web/20231014184810/htt...
13490,2023.MMP1048,2023.MMP1048,Incident,Europe,2023-10-12,2023,October,2.0,NaN,2,5.0,NaN,2.0,NaN,Southern Asia,Vehicle accident / death linked to hazardous t...,Afghanistan,Western Balkans,"Ditch in Kiskunmajsa, Hungary near motorway in...","Roviden, Magyar Nemzet","46.49042299, 19.736968",Eastern Europe,20 éves embercsempész balesetezett Kiskunmajsá...,3.0,https://web.archive.org/web20231014065213/http...
13491,2023.MMP1049,2023.MMP1049,Incident,Mediterranean,2023-10-01,2023,October,NaN,1.0,1,7.0,NaN,NaN,NaN,Western Asia (P),Mixed or unknown,Unknown,Eastern Mediterranean,Unspecified location off the coast of Samandag...,Turkish Coast Guard,"36.07633665, 35.80700626",Uncategorized,Samandağ Sahilinde Başlatılan Arama Kurtarma F...,5.0,https://web.archive.org/web/20231019082127/htt...
13492,2023.MMP1050,2023.MMP1050,Incident,Mediterranean,2023-10-17,2023,October,1.0,NaN,1,37.0,NaN,1.0,NaN,Eastern Africa,Mixed or unknown,Somalia,Eastern Mediterranean,"Unspecified location off the coast of Lesvos, ...","AFP, Enikos","39.30415035, 26.4722731",Uncategorized,Second Deadly Greece Migrant Boat Incident In ...,3.0,https://web.archive.org/web/20231019080934/htt...


In [8]:
# Data profil 

report = sv.analyze(mm_ds_original)
report.show_html('report.html')

                                             |      | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### First incident from 01/06/2014; most recent incident from 17/10/2023

In [5]:
mm_ds_original.shape

(13493, 25)

In [6]:
mm_ds_original.columns

Index(['Main ID', 'Incident ID', 'Incident Type', 'Region of Incident',
       'Website Date', 'Incident year', 'Reported Month', 'Number of Dead',
       'Minimum Estimated Number of Missing',
       'Total Number of Dead and Missing', 'Number of Survivors',
       'Number of Females', 'Number of Males', 'Number of Children',
       'Region of Origin', 'Cause of Death', 'Country of Origin',
       'Migration route', 'Location of death', 'Information Source',
       'Coordinates', 'UNSD Geographical Grouping', 'Article title',
       'Source Quality', 'URL'],
      dtype='object')

In [7]:
# Rename Columns Name for Easy Understandig
renamed_df = mm_ds_original.rename(columns = {'Main ID' : 'main_ID', 'Incident ID' : 'incident_ID', 'Incident Type': 'inc_type', 
                           'Region of Incident' : 'region_inc', 'Website Date': 'website_date',
                           'Incident year' : 'year', 'Reported Month' : 'month', 
                           'Number of Dead' : 'num_dead', 'Minimum Estimated Number of Missing' : 'num_missing',
                           'Total Number of Dead and Missing' : 'total_dead_missing', 'Number of Survivors' : 'num_survivors', 
                           'Number of Females' : 'num_female', 'Number of Males' : 'num_male', 'Number of Children' : 'num_children',
                           'Region of Origin': 'reg_origin', 'Cause of Death' : 'cause_death', 'Country of Origin': 'country_origin',
                           'Migration route':'migration_route', 'Location of death' : 'loc_death', 
                           'Information Source' : 'info_source', 'Coordinates' : 'coordinates', 'UNSD Geographical Grouping' : 'UNSD_geo_group',
                           'Article title': 'article_title', 'Source Quality' : 'source_quality', 'URL': 'url'} 
)
renamed_df.columns

Index(['main_ID', 'incident_ID', 'inc_type', 'region_inc', 'website_date',
       'year', 'month', 'num_dead', 'num_missing', 'total_dead_missing',
       'num_survivors', 'num_female', 'num_male', 'num_children', 'reg_origin',
       'cause_death', 'country_origin', 'migration_route', 'loc_death',
       'info_source', 'coordinates', 'UNSD_geo_group', 'article_title',
       'source_quality', 'url'],
      dtype='object')

# 2. Data Prep: 
## 2.1. Data quality checks

In [8]:
# Count unique values
unique_value_counts = renamed_df.nunique()
print(unique_value_counts)

main_ID               13493
incident_ID           13493
inc_type                  4
region_inc               16
website_date           3186
year                     10
month                    12
num_dead                 85
num_missing             120
total_dead_missing      137
num_survivors           229
num_female               48
num_male                 60
num_children             20
reg_origin               34
cause_death              15
country_origin          345
migration_route          25
loc_death              7768
info_source            3957
coordinates           10004
UNSD_geo_group           19
article_title          4416
source_quality            5
url                    5231
dtype: int64


In [9]:
# Count missing values
na_counts = renamed_df.isna().sum()
print(na_counts)

main_ID                   0
incident_ID               0
inc_type                  0
region_inc                0
website_date             15
year                      0
month                     0
num_dead                599
num_missing           12130
total_dead_missing        0
num_survivors         11308
num_female            11051
num_male               5190
num_children          11871
reg_origin                1
cause_death               0
country_origin           10
migration_route        3126
loc_death                 0
info_source               8
coordinates              36
UNSD_geo_group            1
article_title          8477
source_quality            1
url                    3961
dtype: int64


## 2.2 Consistency checks

In [10]:
renamed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13493 entries, 0 to 13492
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   main_ID             13493 non-null  object 
 1   incident_ID         13493 non-null  object 
 2   inc_type            13493 non-null  object 
 3   region_inc          13493 non-null  object 
 4   website_date        13478 non-null  object 
 5   year                13493 non-null  int64  
 6   month               13493 non-null  object 
 7   num_dead            12894 non-null  float64
 8   num_missing         1363 non-null   float64
 9   total_dead_missing  13493 non-null  int64  
 10  num_survivors       2185 non-null   float64
 11  num_female          2442 non-null   float64
 12  num_male            8303 non-null   float64
 13  num_children        1622 non-null   float64
 14  reg_origin          13492 non-null  object 
 15  cause_death         13493 non-null  object 
 16  coun

In [11]:
renamed_df.dtypes

main_ID                object
incident_ID            object
inc_type               object
region_inc             object
website_date           object
year                    int64
month                  object
num_dead              float64
num_missing           float64
total_dead_missing      int64
num_survivors         float64
num_female            float64
num_male              float64
num_children          float64
reg_origin             object
cause_death            object
country_origin         object
migration_route        object
loc_death              object
info_source            object
coordinates            object
UNSD_geo_group         object
article_title          object
source_quality        float64
url                    object
dtype: object

In [12]:
# Checking whether data frame contains any mixed-type columns
for col in renamed_df.columns.tolist():
  weird = (renamed_df[[col]].applymap(type) != renamed_df[[col]].iloc[0].apply(type)).any(axis = 1) # Test that checks whether the data types within the column are consistent. 
  if len (renamed_df[weird]) > 0: 
    print (col)

website_date
reg_origin
country_origin
migration_route
info_source
coordinates
UNSD_geo_group
article_title
url


### Check these in excel 

In [13]:
string_entries = renamed_df[renamed_df['website_date'].apply(lambda x: isinstance(x, str) and x.isalpha())]
string_entries

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url


In [14]:
numeric_entries = renamed_df[pd.to_numeric(renamed_df['website_date'], errors='coerce').notna()]
numeric_entries

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url


In [15]:
# Sample the data in each column to get a sense of mixed data types
sampled_data = renamed_df.sample(20)  # Change the number to your desired sample size

# Iterate through the columns and display the sampled data
for column in renamed_df.columns:
    print(f"Column: {column}")
    print(sampled_data[column])
    print("\n")

# Check the distribution of unique values in each column
for column in renamed_df.columns:
    print(f"Column: {column}")
    unique_values = renamed_df[column].unique()
    value_counts = renamed_df[column].value_counts()
    print(f"Unique Values: {unique_values}")
    print(f"Value Counts:\n{value_counts}\n")


Column: main_ID
12933      2023.MMP0481
12333      2022.MMP2132
6261      2019.MMP00941
1499      2016.MMP00386
834       2015.MMP00541
4865      2018.MMP01137
12034      2022.MMP1830
1557      2016.MMP00444
10792      2022.MMP0542
6905      2019.MMP01614
4891      2018.MMP01163
2886      2017.MMP00487
10342     2022.MMP00075
3040      2017.MMP00641
12786      2023.MMP0330
1442      2016.MMP00329
5931      2019.MMP00591
2292     2016.MMP007747
12224      2022.MMP2020
11502      2022.MMP1291
Name: main_ID, dtype: object


Column: incident_ID
12933                         2023.MMP0481
12333                         2022.MMP2132
6261                         2019.MMP00941
1499                         2016.MMP00386
834                          2015.MMP00541
4865                         2018.MMP01137
12034                         2022.MMP1830
1557                         2016.MMP00444
10792                         2022.MMP0542
6905                         2019.MMP01614
4891                   

In [16]:
# Transform column "Website date" 

expected_format = r"\d{4}-\d{2}-\d{2}"
renamed_df['website_date'] = renamed_df['website_date'].apply(lambda x: re.search(expected_format, str(x)).group(0) if re.search(expected_format, str(x)) else None)
renamed_df['website_date'] = pd.to_datetime(renamed_df['website_date'], format='%Y-%m-%d', errors='coerce')
date_counts = renamed_df['website_date'].value_counts()
print(date_counts)

website_date
2016-03-20    27
2016-02-21    20
2016-07-31    19
2022-05-17    19
2016-12-19    19
              ..
2017-08-04     1
2017-08-05     1
2017-08-14     1
2017-08-15     1
2023-10-17     1
Name: count, Length: 3186, dtype: int64


In [17]:
unique_dates_count = renamed_df['website_date'].nunique()
print(f"Number of unique dates: {unique_dates_count}")

Number of unique dates: 3186


In [18]:
# Checking whether data frame contains any mixed-type columns
for col in renamed_df.columns.tolist():
  weird = (renamed_df[[col]].applymap(type) != renamed_df[[col]].iloc[0].apply(type)).any(axis = 1) # Test that checks whether the data types within the column are consistent. 
  if len (renamed_df[weird]) > 0: 
    print (col)

website_date
reg_origin
country_origin
migration_route
info_source
coordinates
UNSD_geo_group
article_title
url


In [19]:
string_entries = renamed_df[renamed_df['reg_origin'].apply(lambda x: isinstance(x, str) and x.isalpha())]
string_entries

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url
6,2014.MMP00007,2014.MMP00007,Incident,Mediterranean,2014-01-19,2014,January,12.0,NaN,12,NaN,9.0,NaN,3.0,Unknown,Drowning,"Afghanistan,Syrian Arab Republic",Eastern Mediterranean,Waters near Greece while being towed back to T...,European Council on Refugees and Exiles,"37.2832, 27",Uncategorized,12 refugees die during alleged push-back opera...,5.0,http://bit.ly/2aMCwfg
15,2014.MMP00016,2014.MMP00016,Incident,Caribbean,2014-01-30,2014,January,1.0,NaN,1,NaN,NaN,1.0,1.0,Caribbean,Drowning,Haiti,Caribbean to US,"Off the coast of Fort Lauderdale, Florida",Local10.com,"26.116, -80",Caribbean,Boat with Haitian migrants washes up South Flo...,1.0,http://bit.ly/1zU2LSq
17,2014.MMP00018,2014.MMP00018,Incident,Mediterranean,2014-02-06,2014,February,15.0,0.0,15,0.0,NaN,1.0,NaN,Unknown,Violence,"Cameroon,Unknown",Western Mediterranean,Ceuta waters,L'Espresso,"35.8793, -5",Uncategorized,Migrants clubbed to death at the border Boycot...,1.0,http://bit.ly/U4b8wu
24,2014.MMP00025,2014.MMP00025,Incident,Caribbean,2014-02-20,2014,February,8.0,0.0,8,0.0,NaN,NaN,NaN,Caribbean,Drowning,Haiti,Haiti to Dominican Republic,Dominican Republic,AP,"18.4505, -68",Caribbean,Capsizing death toll rises to 8 Haitians,1.0,http://bit.ly/1sWDTeE
32,2014.MMP00033,2014.MMP00033,Incident,Europe,2014-03-09,2014,March,1.0,0.0,1,0.0,NaN,1.0,NaN,Europe,Violence,Albania,English Channel to the UK,France - Calais (Killed on motorway),"La Voix du Nord, Daily Mail UK","50.9365, 2",Western Europe,Dying to get into Britain...,1.0,https://archive.vn/USuIs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13474,2023.MMP1032,2023.MMP1032,Incident,Central America,2023-10-06,2023,October,1.0,NaN,1,NaN,1.0,NaN,NaN,Caribbean,Harsh environmental conditions / lack of adequ...,Cuba,US-Mexico border crossing,"Undetermined location in Chihuahua, Mexico","El Diario de Chihuahua,Cubita Now","28.65673608, -106.0801912",Central America,Migrante cubana embarazada habría muerto en Mé...,3.0,https://n9.cl/b34mk
13475,2023.MMP1033,2023.MMP1033,Incident,Caribbean,2023-10-07,2023,October,2.0,NaN,2,8.0,NaN,2.0,NaN,Caribbean,Drowning,Cuba,Caribbean to US,"Caribbean Sea, International Waters, near Caym...","Cibercuba,Cubita Now","19.67112691, -81.08434156",Uncategorized,Familia desesperada pide información sobre bal...,3.0,https://n9.cl/p0knl
13482,2023.MMP1040,2023.MMP1040,Incident,Central America,2023-10-10,2023,October,1.0,NaN,1,NaN,1.0,NaN,NaN,Caribbean,Sickness / lack of access to adequate healthcare,Cuba,NaN,"At the height of Municiality of Quimistán, Hon...",Cibercuba,"15.33973661, -88.4001299",Central America,Migrante cubana muere de un infarto dentro de ...,1.0,https://n9.cl/6srjt4
13484,2023.MMP1042,2023.MMP1042,Incident,Central America,2023-10-10,2023,October,1.0,NaN,1,1.0,1.0,NaN,NaN,Caribbean,Harsh environmental conditions / lack of adequ...,Haiti,NaN,"Pijijiapan-Tonalá section, on the coast of Chi...",La Jornada,"16.07664167, -93.76383224",Central America,Muere migrante haitiana en accidente carretero...,1.0,https://n9.cl/hxwv9


In [20]:
# Verify the unique name of the months
# any duplication -->spellings
renamed_df['month'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

In [21]:
renamed_df.describe()

,website_date,year,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,source_quality
count,13478,13493.000000,12894.000000,1363.000000,13493.00000,2185.000000,2442.000000,8303.000000,1622.000000,13492.000000
mean,2019-09-17 07:35:40.554978304,2019.197510,2.619280,18.667645,4.38872,40.068192,2.515971,1.825485,1.956227,3.264231
min,2014-01-02 00:00:00,2014.000000,0.000000,-2.000000,1.00000,-2.000000,0.000000,0.000000,0.000000,1.000000
25%,2017-10-08 00:00:00,2017.000000,1.000000,2.000000,1.00000,4.000000,1.000000,1.000000,1.000000,2.000000
50%,2019-10-05 00:00:00,2019.000000,1.000000,6.000000,1.00000,12.000000,1.000000,1.000000,1.000000,4.000000
75%,2021-12-01 00:00:00,2021.000000,2.000000,17.000000,2.00000,38.000000,2.000000,1.000000,2.000000,4.000000
max,2023-10-17 00:00:00,2023.000000,750.000000,750.000000,1022.00000,1950.000000,94.000000,135.000000,250.000000,5.000000
std,NaN,2.482202,9.612398,45.131106,19.66173,97.035667,6.212009,4.729596,6.606593,1.371832


### Inconsistency: Min = -2 for num_missing, num_survivors

In [22]:
renamed_df.describe(include= object)

,main_ID,incident_ID,inc_type,region_inc,month,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,url
count,13493,13493,13493,13493,13493,13492,13493,13483,10367,13493,13485,13457,13492,5016,9532
unique,13493,13493,4,16,12,34,15,345,25,7768,3957,10004,19,4416,5231
top,2014.MMP00001,2014.MMP00001,Incident,North America,September,Latin America / Caribbean (P),Drowning,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"30.96704016, 61.82069052",Northern America,MHub Survey Data Italy 1.17,http://www.mixedmigration.org/4mi/
freq,1,1,13126,2849,1315,2259,3409,7405,3581,1061,1574,200,2851,103,2730


### Change missings in numeric variables to NaN and in string variables to "Unknown", because these are true missings (information unknown) and not "zero" as in no person dead/missing/survided.

In [23]:
# Create a copy of the original DataFrame
repl_miss_df = renamed_df.copy()

# Replace missing values with NaN for numeric and datetime columns
numeric_datetime_columns = repl_miss_df.select_dtypes(include=['float64', 'int64', 'datetime64[ns]'])
repl_miss_df[numeric_datetime_columns.columns] = numeric_datetime_columns.fillna(pd.NA)

# Replace missing values with "Unknown" for string columns
string_columns = repl_miss_df.select_dtypes(include=['object'])
repl_miss_df[string_columns.columns] = string_columns.fillna("Unknown")

In [24]:
repl_miss_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13493 entries, 0 to 13492
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   main_ID             13493 non-null  object        
 1   incident_ID         13493 non-null  object        
 2   inc_type            13493 non-null  object        
 3   region_inc          13493 non-null  object        
 4   website_date        13478 non-null  datetime64[ns]
 5   year                13493 non-null  int64         
 6   month               13493 non-null  object        
 7   num_dead            12894 non-null  float64       
 8   num_missing         1363 non-null   float64       
 9   total_dead_missing  13493 non-null  int64         
 10  num_survivors       2185 non-null   float64       
 11  num_female          2442 non-null   float64       
 12  num_male            8303 non-null   float64       
 13  num_children        1622 non-null   float64   

In [25]:
na_counts = repl_miss_df.isna().sum()
print(na_counts)

main_ID                   0
incident_ID               0
inc_type                  0
region_inc                0
website_date             15
year                      0
month                     0
num_dead                599
num_missing           12130
total_dead_missing        0
num_survivors         11308
num_female            11051
num_male               5190
num_children          11871
reg_origin                0
cause_death               0
country_origin            0
migration_route           0
loc_death                 0
info_source               0
coordinates               0
UNSD_geo_group            0
article_title             0
source_quality            1
url                       0
dtype: int64


In [26]:
unknown_counts = (repl_miss_df == "Unknown").sum()
unknown_counts

main_ID                  0
incident_ID              0
inc_type                 0
region_inc               0
website_date             0
year                     0
month                    0
num_dead                 0
num_missing              0
total_dead_missing       0
num_survivors            0
num_female               0
num_male                 0
num_children             0
reg_origin            1770
cause_death              0
country_origin        7415
migration_route       3126
loc_death                2
info_source              8
coordinates             36
UNSD_geo_group           1
article_title         8477
source_quality           0
url                   3961
dtype: int64

In [27]:
nan_counts = repl_miss_df[repl_miss_df.isna().any(axis=1)]

In [28]:
unknown_counts = repl_miss_df[repl_miss_df.eq("Unknown").any(axis=1)]

In [29]:
print("NaN Counts:")
print(nan_counts)
print("\nUnknown Counts:")
print(unknown_counts)

NaN Counts:
             main_ID    incident_ID  inc_type     region_inc website_date  \
0      2014.MMP00001  2014.MMP00001  Incident  North America   2014-01-06   
1      2014.MMP00002  2014.MMP00002  Incident  North America   2014-01-12   
2      2014.MMP00003  2014.MMP00003  Incident  North America   2014-01-14   
3      2014.MMP00004  2014.MMP00004  Incident  North America   2014-01-16   
4      2014.MMP00005  2014.MMP00005  Incident         Europe   2014-01-16   
...              ...            ...       ...            ...          ...   
13488   2023.MMP1046   2023.MMP1046  Incident  North America   2023-10-11   
13489   2023.MMP1047   2023.MMP1047  Incident         Europe   2023-10-13   
13490   2023.MMP1048   2023.MMP1048  Incident         Europe   2023-10-12   
13491   2023.MMP1049   2023.MMP1049  Incident  Mediterranean   2023-10-01   
13492   2023.MMP1050   2023.MMP1050  Incident  Mediterranean   2023-10-17   

       year    month  num_dead  num_missing  total_dead_missing

## Now, check again for inconsistent types

In [30]:
# Checking whether data frame contains any mixed-type columns
for col in repl_miss_df.columns.tolist():
  weird = (repl_miss_df[[col]].applymap(type) != repl_miss_df[[col]].iloc[0].apply(type)).any(axis = 1) # Test that checks whether the data types within the column are consistent. 
  if len (repl_miss_df[weird]) > 0: 
    print (col)

website_date


In [31]:
# Get the data types of the underlying data in the timestamp column
data_types = repl_miss_df['website_date'].apply(lambda x: type(x))

# Get a set of unique data types present in the column
unique_data_types = set(data_types)

# Print the unique data types
print("Unique data types in 'website_date':")
for data_type in unique_data_types:
    print(data_type)

Unique data types in 'website_date':
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.nattype.NaTType'>


In [32]:
# Count the number of missing values (NaT) in the 'Website Date' column
missing_values_count = repl_miss_df['website_date'].isna().sum()

print(f"Number of missing values in 'website_date': {missing_values_count}")

Number of missing values in 'website_date': 15


### Leave as Is: I don't have enough information to impute or estimate them accurately.

In [33]:
# Check summary stats again

summary_stats = repl_miss_df.describe()
summary_stats

,website_date,year,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,source_quality
count,13478,13493.000000,12894.000000,1363.000000,13493.00000,2185.000000,2442.000000,8303.000000,1622.000000,13492.000000
mean,2019-09-17 07:35:40.554978304,2019.197510,2.619280,18.667645,4.38872,40.068192,2.515971,1.825485,1.956227,3.264231
min,2014-01-02 00:00:00,2014.000000,0.000000,-2.000000,1.00000,-2.000000,0.000000,0.000000,0.000000,1.000000
25%,2017-10-08 00:00:00,2017.000000,1.000000,2.000000,1.00000,4.000000,1.000000,1.000000,1.000000,2.000000
50%,2019-10-05 00:00:00,2019.000000,1.000000,6.000000,1.00000,12.000000,1.000000,1.000000,1.000000,4.000000
75%,2021-12-01 00:00:00,2021.000000,2.000000,17.000000,2.00000,38.000000,2.000000,1.000000,2.000000,4.000000
max,2023-10-17 00:00:00,2023.000000,750.000000,750.000000,1022.00000,1950.000000,94.000000,135.000000,250.000000,5.000000
std,NaN,2.482202,9.612398,45.131106,19.66173,97.035667,6.212009,4.729596,6.606593,1.371832


In [34]:
# Print for report:

summary_stats.to_excel("summary_output.xlsx")

In [35]:
# Filter rows with a minus sign in the 'num_missing' column
negative_values = repl_miss_df[repl_miss_df['num_missing'] < 0]

# Display the filtered rows
print(negative_values)

             main_ID                                        incident_ID  \
601    2015.MMP00308                                      2015.MMP00308   
806    2015.MMP00513  2015.MMP00513541,2015.MMP00513542,2015.MMP0051...   
858    2015.MMP00565                  2015.MMP00565596,2015.MMP00565597   
10422  2022.MMP00157                                      2022.MMP00157   

                  inc_type       region_inc website_date  year    month  \
601    Cumulative Incident  Central America   2015-08-18  2015   August   
806         Split Incident    Mediterranean   2015-10-16  2015  October   
858         Split Incident    Mediterranean   2015-10-28  2015  October   
10422             Incident    Mediterranean   2022-03-12  2022    March   

       num_dead  num_missing  total_dead_missing  num_survivors  num_female  \
601        20.0         -1.0                  19            1.0         NaN   
806        15.0         -2.0                  13            NaN         6.0   
858        

In [36]:
# Filter rows with a minus sign in the 'num_survivors' column
negative_values = repl_miss_df[repl_miss_df['num_survivors'] < 0]

# Display the filtered rows
print(negative_values)

            main_ID                        incident_ID        inc_type  \
1247  2016.MMP00134  2016.MMP00134134,2016.MMP00134135  Split Incident   
7794  2020.MMP00661                      2020.MMP00661        Incident   
7796  2020.MMP00663                      2020.MMP00663        Incident   
7800  2020.MMP00667                      2020.MMP00667        Incident   
7889  2020.MMP00773                      2020.MMP00773        Incident   

              region_inc website_date  year      month  num_dead  num_missing  \
1247     Northern Africa   2016-02-07  2016   February       1.0         56.0   
7794  South-eastern Asia   2020-09-09  2020  September       1.0          NaN   
7796  South-eastern Asia   2020-09-10  2020  September       1.0          NaN   
7800  South-eastern Asia   2020-09-11  2020  September       1.0          NaN   
7889     Northern Africa   2020-10-03  2020    October       2.0          NaN   

      total_dead_missing  num_survivors  num_female  num_male  num_c

In [37]:
# Create a copy of the original DataFrame
incon_check_df = repl_miss_df.copy()

# Change negative values to positive in "num_missing" and "num_survivors" columns
incon_check_df['num_missing'] = incon_check_df['num_missing'].abs()
incon_check_df['num_survivors'] = incon_check_df['num_survivors'].abs()

In [38]:
incon_check_df.describe()

,website_date,year,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,source_quality
count,13478,13493.000000,12894.000000,1363.000000,13493.00000,2185.000000,2442.000000,8303.000000,1622.000000,13492.000000
mean,2019-09-17 07:35:40.554978304,2019.197510,2.619280,18.674982,4.38872,40.073684,2.515971,1.825485,1.956227,3.264231
min,2014-01-02 00:00:00,2014.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2017-10-08 00:00:00,2017.000000,1.000000,2.000000,1.00000,4.000000,1.000000,1.000000,1.000000,2.000000
50%,2019-10-05 00:00:00,2019.000000,1.000000,6.000000,1.00000,12.000000,1.000000,1.000000,1.000000,4.000000
75%,2021-12-01 00:00:00,2021.000000,2.000000,17.000000,2.00000,38.000000,2.000000,1.000000,2.000000,4.000000
max,2023-10-17 00:00:00,2023.000000,750.000000,750.000000,1022.00000,1950.000000,94.000000,135.000000,250.000000,5.000000
std,NaN,2.482202,9.612398,45.128069,19.66173,97.033398,6.212009,4.729596,6.606593,1.371832


In [39]:
incon_check_df.head(30)

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url
0,2014.MMP00001,2014.MMP00001,Incident,North America,2014-01-06,2014,January,1.0,NaN,1,NaN,NaN,1.0,NaN,Central America,Mixed or unknown,Guatemala,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.650259, -110.366453",Northern America,Unknown,5.0,http://humaneborders.info/
1,2014.MMP00002,2014.MMP00002,Incident,North America,2014-01-12,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.59713, -111.73756",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown
2,2014.MMP00003,2014.MMP00003,Incident,North America,2014-01-14,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.94026, -113.01125",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown
3,2014.MMP00004,2014.MMP00004,Incident,North America,2014-01-16,2014,January,1.0,NaN,1,NaN,NaN,1.0,NaN,Central America,Violence,Mexico,US-Mexico border crossing,"near Douglas, Arizona, USA","Ministry of Foreign Affairs Mexico, Pima Count...","31.506777, -109.315632",Northern America,Mexico condemns the use of force by a border p...,5.0,http://bit.ly/1qfIw00
4,2014.MMP00005,2014.MMP00005,Incident,Europe,2014-01-16,2014,January,1.0,0.0,1,2.0,NaN,1.0,NaN,Northern Africa,Harsh environmental conditions / lack of adequ...,Sudan,Unknown,Border between Russia and Estonia,EUBusiness (Agence France-Presse),"59.1551, 28",Northern Europe,Sudanese migrant freezes to death in illegal E...,1.0,http://bit.ly/1rTFTjR
5,2014.MMP00006,2014.MMP00006,Incident,North America,2014-01-17,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Violence,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"32.45435, -113.18402",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown
6,2014.MMP00007,2014.MMP00007,Incident,Mediterranean,2014-01-19,2014,January,12.0,NaN,12,NaN,9.0,NaN,3.0,Unknown,Drowning,"Afghanistan,Syrian Arab Republic",Eastern Mediterranean,Waters near Greece while being towed back to T...,European Council on Refugees and Exiles,"37.2832, 27",Uncategorized,12 refugees die during alleged push-back opera...,5.0,http://bit.ly/2aMCwfg
7,2014.MMP00008,2014.MMP00008,Incident,North America,2014-01-22,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"32.478317, -113.182833",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown
8,2014.MMP00009,2014.MMP00009,Incident,North America,2014-01-24,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.81154, -111.01101",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown
9,2014.MMP00010,2014.MMP00010,Incident,North America,2014-01-24,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"32.174017, -112.174583",Northern America,Undocumented Border Crosse

In [40]:
# Checks

# Calculate the sum of values in the "old_num_missing" column
column_sum = repl_miss_df['num_missing'].sum()

print(f"Sum of values in 'old_num_missing': {column_sum}")

# Calculate the sum of values in the "new_num_missing" column
column_sum = incon_check_df['num_missing'].sum()

print(f"Sum of values in 'new_num_missing': {column_sum}")


# Calculate the sum of values in the "old_num_survivors" column
column_sum = repl_miss_df['num_survivors'].sum()

print(f"Sum of values in 'old_num_survivors': {column_sum}")

# Calculate the sum of values in the "num_survivors" column
column_sum = incon_check_df['num_survivors'].sum()

print(f"Sum of values in 'new_num_survivors': {column_sum}")


Sum of values in 'old_num_missing': 25444.0
Sum of values in 'new_num_missing': 25454.0
Sum of values in 'old_num_survivors': 87549.0
Sum of values in 'new_num_survivors': 87561.0


In [41]:
incon_check_df.shape

(13493, 25)

In [42]:
# Check num_missing old df

row_indices = [601, 806, 858, 10422]
selected_rows = repl_miss_df.iloc[row_indices]
selected_rows

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url
601,2015.MMP00308,2015.MMP00308,Cumulative Incident,Central America,2015-08-18,2015,August,20.0,-1.0,19,1.0,NaN,1.0,1.0,Central America,Mixed or unknown,Unknown,Unknown,"State of Puebla, Mexico",El Popular,"18.082037, -97.920056",Central America,Acusan a policías de Edomex por violación de m...,3.0,https://archive.vn/wip/wDvO3
806,2015.MMP00513,"2015.MMP00513541,2015.MMP00513542,2015.MMP0051...",Split Incident,Mediterranean,2015-10-16,2015,October,15.0,-2.0,13,NaN,6.0,3.0,6.0,Western Asia,"Drowning,Mixed or unknown","Afghanistan,Syrian Arab Republic,Unknown",Eastern Mediterranean,"Found on the coast of Lesvos,Off the coast of ...",Hurriyet Daily News,"39.4162, 26.4074",Uncategorized,Twelve migrants drown after boat sinks off of ...,1.0,"http://bit.ly/1RnKlDw,http://www.hcg.gr/node/1..."
858,2015.MMP00565,"2015.MMP00565596,2015.MMP00565597",Split Incident,Mediterranean,2015-10-28,2015,October,57.0,-1.0,56,274.0,7.0,17.0,20.0,Western / Southern Asia (P),Drowning,Unknown,Eastern Mediterranean,"Found on beaches in Lesvos,north of Lesvos, Gr...",Hellenic Coast Guard,"39.3856462, 26.216722",Uncategorized,Unknown,5.0,http://www.hcg.gr/node/11414
10422,2022.MMP00157,2022.MMP00157,Incident,Mediterranean,2022-03-12,2022,March,2.0,-1.0,1,NaN,NaN,2.0,NaN,Unknown,Drowning,"Egypt,Unknown",Central Mediterranean,"Remains washed ashore at Surman beach, Libya",IOM Libya,"32.79560599, 12.583315",Uncategorized,Unknown,4.0,Unknown


In [43]:
# Check num_missing new df

row_indices = [601, 806, 858, 10422]
selected_rows = incon_check_df.iloc[row_indices]
selected_rows

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url
601,2015.MMP00308,2015.MMP00308,Cumulative Incident,Central America,2015-08-18,2015,August,20.0,1.0,19,1.0,NaN,1.0,1.0,Central America,Mixed or unknown,Unknown,Unknown,"State of Puebla, Mexico",El Popular,"18.082037, -97.920056",Central America,Acusan a policías de Edomex por violación de m...,3.0,https://archive.vn/wip/wDvO3
806,2015.MMP00513,"2015.MMP00513541,2015.MMP00513542,2015.MMP0051...",Split Incident,Mediterranean,2015-10-16,2015,October,15.0,2.0,13,NaN,6.0,3.0,6.0,Western Asia,"Drowning,Mixed or unknown","Afghanistan,Syrian Arab Republic,Unknown",Eastern Mediterranean,"Found on the coast of Lesvos,Off the coast of ...",Hurriyet Daily News,"39.4162, 26.4074",Uncategorized,Twelve migrants drown after boat sinks off of ...,1.0,"http://bit.ly/1RnKlDw,http://www.hcg.gr/node/1..."
858,2015.MMP00565,"2015.MMP00565596,2015.MMP00565597",Split Incident,Mediterranean,2015-10-28,2015,October,57.0,1.0,56,274.0,7.0,17.0,20.0,Western / Southern Asia (P),Drowning,Unknown,Eastern Mediterranean,"Found on beaches in Lesvos,north of Lesvos, Gr...",Hellenic Coast Guard,"39.3856462, 26.216722",Uncategorized,Unknown,5.0,http://www.hcg.gr/node/11414
10422,2022.MMP00157,2022.MMP00157,Incident,Mediterranean,2022-03-12,2022,March,2.0,1.0,1,NaN,NaN,2.0,NaN,Unknown,Drowning,"Egypt,Unknown",Central Mediterranean,"Remains washed ashore at Surman beach, Libya",IOM Libya,"32.79560599, 12.583315",Uncategorized,Unknown,4.0,Unknown


In [44]:
# Check num_survivors old df
row_indices = [1247, 7794, 7796, 7800, 7889]
selected_rows = repl_miss_df.iloc[row_indices]
selected_rows

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url
1247,2016.MMP00134,"2016.MMP00134134,2016.MMP00134135",Split Incident,Northern Africa,2016-02-07,2016,February,1.0,56.0,57,-1.0,NaN,1.0,NaN,Sub-Saharan Africa (P),"Drowning,Harsh environmental conditions / lack...",Unknown,Western Africa / Atlantic route to the Canary ...,"Between Dakhla, Western Sahara and Gran Canari...",Asociación Pro Derechos Humanos de Andalucía,"26.9455294, -15.5943805",Uncategorized,Salvamento reanuda la búsqueda de la patera qu...,3.0,"http://bit.ly/1SLpxdH,http://bit.ly/2qSWQ3s"
7794,2020.MMP00661,2020.MMP00661,Incident,South-eastern Asia,2020-09-09,2020,September,1.0,NaN,1,-1.0,1.0,NaN,NaN,Southern Asia,Sickness / lack of access to adequate healthcare,Myanmar,Unknown,"In Lhoksemawe hospital, Aceh province, Indones...","EFE, UNHCR","5.1811638, 97.1413222",South-eastern Asia,Tres refugiados rohinyás mueren en Indonesia t...,4.0,https://archive.ph/iglFk
7796,2020.MMP00663,2020.MMP00663,Incident,South-eastern Asia,2020-09-10,2020,September,1.0,NaN,1,-1.0,NaN,1.0,NaN,Southern Asia,Sickness / lack of access to adequate healthcare,Myanmar,Unknown,"In Lhoksemawe hospital, Aceh province, Indones...","EFE, UNHCR","5.1811638, 97.1413222",South-eastern Asia,Tres refugiados rohinyás mueren en Indonesia t...,4.0,https://archive.ph/iglFk
7800,2020.MMP00667,2020.MMP00667,Incident,South-eastern Asia,2020-09-11,2020,September,1.0,NaN,1,-1.0,1.0,NaN,NaN,Southern Asia,Sickness / lack of access to adequate healthcare,Myanmar,Unknown,"In Lhoksemawe hospital, Aceh province, Indones...","EFE, UNHCR","5.1811638, 97.1413222",South-eastern Asia,Tres refugiados rohinyás mueren en Indonesia t...,4.0,https://archive.ph/iglFk
7889,2020.MMP00773,2020.MMP00773,Incident,Northern Africa,2020-10-03,2020,October,2.0,NaN,2,-2.0,NaN,2.0,NaN,Sub-Saharan Africa (P),Harsh environmental conditions / lack of adequ...,Unknown,Western Africa / Atlantic route to the Canary ...,"In a hospital in Las Palmas, Spain - Rescued t...","El Diario, Txema Santana","28.1008837, -15.4653897",Uncategorized,Mueren este sábado dos migrantes que se encont...,3.0,https://archive.vn/4TntV


In [45]:
# Check num_survivors new df

row_indices = [1247, 7794, 7796, 7800, 7889]
selected_rows = incon_check_df.iloc[row_indices]
selected_rows

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url
1247,2016.MMP00134,"2016.MMP00134134,2016.MMP00134135",Split Incident,Northern Africa,2016-02-07,2016,February,1.0,56.0,57,1.0,NaN,1.0,NaN,Sub-Saharan Africa (P),"Drowning,Harsh environmental conditions / lack...",Unknown,Western Africa / Atlantic route to the Canary ...,"Between Dakhla, Western Sahara and Gran Canari...",Asociación Pro Derechos Humanos de Andalucía,"26.9455294, -15.5943805",Uncategorized,Salvamento reanuda la búsqueda de la patera qu...,3.0,"http://bit.ly/1SLpxdH,http://bit.ly/2qSWQ3s"
7794,2020.MMP00661,2020.MMP00661,Incident,South-eastern Asia,2020-09-09,2020,September,1.0,NaN,1,1.0,1.0,NaN,NaN,Southern Asia,Sickness / lack of access to adequate healthcare,Myanmar,Unknown,"In Lhoksemawe hospital, Aceh province, Indones...","EFE, UNHCR","5.1811638, 97.1413222",South-eastern Asia,Tres refugiados rohinyás mueren en Indonesia t...,4.0,https://archive.ph/iglFk
7796,2020.MMP00663,2020.MMP00663,Incident,South-eastern Asia,2020-09-10,2020,September,1.0,NaN,1,1.0,NaN,1.0,NaN,Southern Asia,Sickness / lack of access to adequate healthcare,Myanmar,Unknown,"In Lhoksemawe hospital, Aceh province, Indones...","EFE, UNHCR","5.1811638, 97.1413222",South-eastern Asia,Tres refugiados rohinyás mueren en Indonesia t...,4.0,https://archive.ph/iglFk
7800,2020.MMP00667,2020.MMP00667,Incident,South-eastern Asia,2020-09-11,2020,September,1.0,NaN,1,1.0,1.0,NaN,NaN,Southern Asia,Sickness / lack of access to adequate healthcare,Myanmar,Unknown,"In Lhoksemawe hospital, Aceh province, Indones...","EFE, UNHCR","5.1811638, 97.1413222",South-eastern Asia,Tres refugiados rohinyás mueren en Indonesia t...,4.0,https://archive.ph/iglFk
7889,2020.MMP00773,2020.MMP00773,Incident,Northern Africa,2020-10-03,2020,October,2.0,NaN,2,2.0,NaN,2.0,NaN,Sub-Saharan Africa (P),Harsh environmental conditions / lack of adequ...,Unknown,Western Africa / Atlantic route to the Canary ...,"In a hospital in Las Palmas, Spain - Rescued t...","El Diario, Txema Santana","28.1008837, -15.4653897",Uncategorized,Mueren este sábado dos migrantes que se encont...,3.0,https://archive.vn/4TntV


### Calculate the new sum of the "total_dead_missing" column
### Consideration: adding a column to another column when there are NaNs will result in NaNs in the new column

In [46]:
incon_check_df['NaN_rep_tot_dead_missing'] = incon_check_df['num_dead'].add(incon_check_df['num_missing'], fill_value=0)

In [47]:
incon_check_df.head(30)

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url,NaN_rep_tot_dead_missing
0,2014.MMP00001,2014.MMP00001,Incident,North America,2014-01-06,2014,January,1.0,NaN,1,NaN,NaN,1.0,NaN,Central America,Mixed or unknown,Guatemala,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.650259, -110.366453",Northern America,Unknown,5.0,http://humaneborders.info/,1.0
1,2014.MMP00002,2014.MMP00002,Incident,North America,2014-01-12,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.59713, -111.73756",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown,1.0
2,2014.MMP00003,2014.MMP00003,Incident,North America,2014-01-14,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.94026, -113.01125",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown,1.0
3,2014.MMP00004,2014.MMP00004,Incident,North America,2014-01-16,2014,January,1.0,NaN,1,NaN,NaN,1.0,NaN,Central America,Violence,Mexico,US-Mexico border crossing,"near Douglas, Arizona, USA","Ministry of Foreign Affairs Mexico, Pima Count...","31.506777, -109.315632",Northern America,Mexico condemns the use of force by a border p...,5.0,http://bit.ly/1qfIw00,1.0
4,2014.MMP00005,2014.MMP00005,Incident,Europe,2014-01-16,2014,January,1.0,0.0,1,2.0,NaN,1.0,NaN,Northern Africa,Harsh environmental conditions / lack of adequ...,Sudan,Unknown,Border between Russia and Estonia,EUBusiness (Agence France-Presse),"59.1551, 28",Northern Europe,Sudanese migrant freezes to death in illegal E...,1.0,http://bit.ly/1rTFTjR,1.0
5,2014.MMP00006,2014.MMP00006,Incident,North America,2014-01-17,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Violence,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"32.45435, -113.18402",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown,1.0
6,2014.MMP00007,2014.MMP00007,Incident,Mediterranean,2014-01-19,2014,January,12.0,NaN,12,NaN,9.0,NaN,3.0,Unknown,Drowning,"Afghanistan,Syrian Arab Republic",Eastern Mediterranean,Waters near Greece while being towed back to T...,European Council on Refugees and Exiles,"37.2832, 27",Uncategorized,12 refugees die during alleged push-back opera...,5.0,http://bit.ly/2aMCwfg,12.0
7,2014.MMP00008,2014.MMP00008,Incident,North America,2014-01-22,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"32.478317, -113.182833",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown,1.0
8,2014.MMP00009,2014.MMP00009,Incident,North America,2014-01-24,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"31.81154, -111.01101",Northern America,Undocumented Border Crossers 01/01/2014-08/29/...,5.0,Unknown,1.0
9,2014.MMP00010,2014.MMP00010,Incident,North America,2014-01-24,2014,January,1.0,NaN,1,NaN,NaN,NaN,NaN,Latin America / Caribbean (P),Mixed or unknown,Unknown,US-Mexico border crossing,Pima Country Office of the Medical Examiner ju...,Pima County Office of the Medical Examiner (PC...,"32.17

In [48]:
# Calculate the sum of values in the "NaN_rep_tot_dead_missing" column
column_sum = incon_check_df['NaN_rep_tot_dead_missing'].sum()

print(f"Sum of values in 'NaN_rep_tot_dead_missing': {column_sum}")

Sum of values in 'NaN_rep_tot_dead_missing': 59227.0


### Fits exactly: + 10 (absolute sum of negative values in "num_missing" = 5; transform to positive results in absolute sum *2)

In [49]:
summary_stats_2 = incon_check_df.describe()
summary_stats_2.to_excel("summary_output.xlsx")
summary_stats_2

,website_date,year,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,source_quality,NaN_rep_tot_dead_missing
count,13478,13493.000000,12894.000000,1363.000000,13493.00000,2185.000000,2442.000000,8303.000000,1622.000000,13492.000000,13493.000000
mean,2019-09-17 07:35:40.554978304,2019.197510,2.619280,18.674982,4.38872,40.073684,2.515971,1.825485,1.956227,3.264231,4.389461
min,2014-01-02 00:00:00,2014.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,2017-10-08 00:00:00,2017.000000,1.000000,2.000000,1.00000,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000
50%,2019-10-05 00:00:00,2019.000000,1.000000,6.000000,1.00000,12.000000,1.000000,1.000000,1.000000,4.000000,1.000000
75%,2021-12-01 00:00:00,2021.000000,2.000000,17.000000,2.00000,38.000000,2.000000,1.000000,2.000000,4.000000,2.000000
max,2023-10-17 00:00:00,2023.000000,750.000000,750.000000,1022.00000,1950.000000,94.000000,135.000000,250.000000,5.000000,1022.000000
std,NaN,2.482202,9.612398,45.128069,19.66173,97.033398,6.212009,4.729596,6.606593,1.371832,19.662386


## 3. Outlier check: search rows with maximum values, check source quality

In [50]:
# Find the maximum value in each numeric column
max_values = incon_check_df.select_dtypes(include=[int, float]).max()

# Find the indices (row numbers) where the maximum values occur
max_indices = {col: incon_check_df[incon_check_df[col] == max_val].index.tolist() for col, max_val in max_values.items()}

# Display the maximum values and their corresponding indices
# List of columns to exclude from the print
exclude_columns = ['year', 'source_quality']


print("Maximum Values (Excluding 'year' and 'source_quality' columns):")
print(max_values.drop(exclude_columns))
print("\nIndices of Rows with Maximum Values:")


max_indices_excluded = {col: indices for col, indices in max_indices.items() if col not in exclude_columns}
print(max_indices_excluded)

Maximum Values (Excluding 'year' and 'source_quality' columns):
num_dead                     750.0
num_missing                  750.0
total_dead_missing          1022.0
num_survivors               1950.0
num_female                    94.0
num_male                     135.0
num_children                 250.0
NaN_rep_tot_dead_missing    1022.0
dtype: float64

Indices of Rows with Maximum Values:
{'num_dead': [401], 'num_missing': [253], 'total_dead_missing': [401], 'num_survivors': [1647], 'num_female': [12329], 'num_male': [2185], 'num_children': [253], 'NaN_rep_tot_dead_missing': [401]}


In [51]:
# Check rows for columns num_dead, num_missing, total_dead_missing , num_survivors, num_children
row_indices = [401, 253, 401, 1647, 12329, 2185, 253]
selected_rows = incon_check_df.iloc[row_indices]
selected_rows

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url,NaN_rep_tot_dead_missing
401,2015.MMP00108,2015.MMP00108,Incident,Mediterranean,2015-04-18,2015,April,750.0,272.0,1022,28.0,NaN,NaN,NaN,Mixed,Drowning,"Algeria,Bangladesh,Côte d'Ivoire,Egypt,Eritrea...",Central Mediterranean,"120 miles south of Lampedusa, 110 kilometers n...",Italian Navy via UNHCR,"35.03, 14.6777",Uncategorized,Migrant boat captain arrested as survivors of ...,5.0,http://bit.ly/1Oa2qYi,1022.0
253,2014.MMP00259,2014.MMP00259,Cumulative Incident,South-eastern Asia,2014-12-31,2014,December,NaN,750.0,750,NaN,NaN,NaN,250.0,South-eastern Asia,Mixed or unknown,Unknown,Unknown,"Departed from Arakan, Myanmar. Vast majority g...",UNHCR,"14.60759586, 93.21449103",South-eastern Asia,Irregular Maritime Movements in South-Eastern ...,4.0,http://bit.ly/1G6YUEZ,750.0
401,2015.MMP00108,2015.MMP00108,Incident,Mediterranean,2015-04-18,2015,April,750.0,272.0,1022,28.0,NaN,NaN,NaN,Mixed,Drowning,"Algeria,Bangladesh,Côte d'Ivoire,Egypt,Eritrea...",Central Mediterranean,"120 miles south of Lampedusa, 110 kilometers n...",Italian Navy via UNHCR,"35.03, 14.6777",Uncategorized,Migrant boat captain arrested as survivors of ...,5.0,http://bit.ly/1Oa2qYi,1022.0
1647,2016.MMP00534,2016.MMP00534,Incident,Mediterranean,2016-06-09,2016,June,2.0,NaN,2,1950.0,NaN,NaN,NaN,Unknown,Mixed or unknown,Unknown,Central Mediterranean,"Between Italy and Libya, likely near the coast...",Reuters,"33.9413036, 12.7194955",Uncategorized,"3,000 migrants rescued off Italian coast; two ...",1.0,http://reut.rs/1YceQ4M,2.0
12329,2022.MMP2128,2022.MMP2128,Incident,Western Asia,2022-02-04,2022,February,160.0,NaN,160,70.0,94.0,66.0,NaN,Eastern Africa (P),Violence,Unknown,Horn of Africa to Yemen crossing,"In the mountains of Ar Raqw and Al-Ghar, Monab...",OHCHR Special Procedures letter,"17.33322184, 43.35179",Western Asia,Ref.: AL SAU 9/2026,4.0,https://archive.ph/FPE1j,160.0
2185,2016.MMP007640,2016.MMP007640,Incident,Mediterranean,2016-11-14,2016,November,NaN,135.0,135,15.0,NaN,135.0,NaN,Sub-Saharan Africa (P),Drowning,Unknown,Central Mediterranean,Unspecified location between Libya and Italy,IOM Italy,"34.2533716, 14.4988153",Uncategorized,Unknown,4.0,http://bit.ly/2eZjnox,135.0
253,2014.MMP00259,2014.MMP00259,Cumulative Incident,South-eastern Asia,2014-12-31,2014,December,NaN,750.0,750,NaN,NaN,NaN,250.0,South-eastern Asia,Mixed or unknown,Unknown,Unknown,"Departed from Arakan, Myanmar. Vast majority g...",UNHCR,"14.60759586, 93.21449103",South-eastern Asia,Irregular Maritime Movements in South-Eastern ...,4.0,http://bit.ly/1G6YUEZ,750.0


## 4. Duplicates check

In [52]:
# Check Duplicates

# Look for full duplicates
df_dups = incon_check_df[incon_check_df.duplicated()]
df_dups

,main_ID,incident_ID,inc_type,region_inc,website_date,year,month,num_dead,num_missing,total_dead_missing,num_survivors,num_female,num_male,num_children,reg_origin,cause_death,country_origin,migration_route,loc_death,info_source,coordinates,UNSD_geo_group,article_title,source_quality,url,NaN_rep_tot_dead_missing


### No full duplicates.

## 5. Timelineness

In [53]:
print('The Missing Migrants dataset covers the period {0} to {1}'.format(str(incon_check_df['website_date'][13492]), str(incon_check_df['website_date'][0])))

The Missing Migrants dataset covers the period 2023-10-17 00:00:00 to 2014-01-06 00:00:00


## 6. Data Profile

In [54]:
profile_numeric = incon_check_df.describe()
profile_numeric.to_excel("prof_num_output.xlsx")

In [55]:
profile_objects = incon_check_df.describe(include= object)
profile_objects.to_excel("prof_str_output.xlsx")

In [56]:
incon_check_df.shape

(13493, 26)

## 7. Further inconsistency check:
## Check unique values in string variables 

In [57]:
# List of column names for which you want unique values
object_columns = ['inc_type', 'region_inc', 'reg_origin', 'cause_death', 'country_origin', 'migration_route', 'loc_death', 'info_source', 'UNSD_geo_group']

# Get unique values for each column of interest
unique_values = {}

for column in object_columns:
    unique_values[column] = incon_check_df[column].unique()

# Print unique values for each column of interest
for column, values in unique_values.items():
    print(f"Unique values for {column}:")
    print(values)

Unique values for inc_type:
['Incident' 'Cumulative Incident' 'Split Incident'
 'Incident,Split Incident']
Unique values for region_inc:
['North America' 'Europe' 'Mediterranean' 'Caribbean' 'South-eastern Asia'
 'Southern Asia' 'Eastern Africa' 'Central America' 'Northern Africa'
 'Eastern Asia' 'Western Asia' 'Western Africa' 'South America'
 'Southern Africa' 'Middle Africa' 'Central Asia']
Unique values for reg_origin:
['Central America' 'Latin America / Caribbean (P)' 'Northern Africa'
 'Unknown' 'Southern Asia' 'Caribbean' 'South-eastern Asia'
 'Eastern Africa' 'Europe' 'South America' 'Western Asia' 'Middle Africa'
 'Eastern Asia' 'Sub-Saharan Africa (P)' 'Western Africa' 'Mixed'
 'Western / Southern Asia (P)' 'Eastern Africa (P)'
 'Western / Southern Asia' 'Eastern Asia (P)' 'Western Africa (P)'
 'Sub-Saharan Africa' 'Oceania' 'Central Asia' 'Southern Asia (P)'
 'Northern Africa (P)' 'Southern Africa' 'Caribbean (P)'
 'Western Asia (P)' 'South America (P)' 'Central America (P)'

## No further inconsistencies. 

In [59]:
# Export data to pkl
incon_check_df.to_pickle(os.path.join(path, 'Data', 'prepared data', 'prep_mm_data.pkl'))